In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('..\\data\\raw\\gps_traces\\taxi_data_subset.csv',  )

In [3]:
df.head()

,index,DriveNo,Date and Time,Longitude,Latitude
0,1,156,2014-02-01 00:00:00.739166+01,41.883672,12.487778
1,2,187,2014-02-01 00:00:01.148457+01,41.928543,12.469037
2,3,297,2014-02-01 00:00:01.220066+01,41.891069,12.492705
3,4,89,2014-02-01 00:00:01.470854+01,41.793177,12.432122
4,5,79,2014-02-01 00:00:01.631136+01,41.900275,12.462746


In [4]:
df.drop(columns = ['index'], inplace= True)

In [5]:
df = df.rename(columns={
    "DriveNo": "drive_id",
    "Date and Time": "timestamp",
    "Longitude": "lon",
    "Latitude": "lat"
})

In [6]:
df.head()

,drive_id,timestamp,lon,lat
0,156,2014-02-01 00:00:00.739166+01,41.883672,12.487778
1,187,2014-02-01 00:00:01.148457+01,41.928543,12.469037
2,297,2014-02-01 00:00:01.220066+01,41.891069,12.492705
3,89,2014-02-01 00:00:01.470854+01,41.793177,12.432122
4,79,2014-02-01 00:00:01.631136+01,41.900275,12.462746


In [7]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True)

In [8]:
df.head()

,drive_id,timestamp,lon,lat
0,156,2014-01-31 23:00:00.739166+00:00,41.883672,12.487778
1,187,2014-01-31 23:00:01.148457+00:00,41.928543,12.469037
2,297,2014-01-31 23:00:01.220066+00:00,41.891069,12.492705
3,89,2014-01-31 23:00:01.470854+00:00,41.793177,12.432122
4,79,2014-01-31 23:00:01.631136+00:00,41.900275,12.462746


In [9]:
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lon'] = pd.to_numeric(df['lon'], errors='coerce')

In [10]:
df.head()

,drive_id,timestamp,lon,lat
0,156,2014-01-31 23:00:00.739166+00:00,41.883672,12.487778
1,187,2014-01-31 23:00:01.148457+00:00,41.928543,12.469037
2,297,2014-01-31 23:00:01.220066+00:00,41.891069,12.492705
3,89,2014-01-31 23:00:01.470854+00:00,41.793177,12.432122
4,79,2014-01-31 23:00:01.631136+00:00,41.900275,12.462746


In [11]:
df = df[(df['lat'] >= -90) & (df['lat'] <= 90)]
df = df[(df['lon'] >= -180) & (df['lon'] <=180)]

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   drive_id   10000 non-null  int64              
 1   timestamp  10000 non-null  datetime64[ns, UTC]
 2   lon        10000 non-null  float64            
 3   lat        10000 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1)
memory usage: 312.6 KB


In [13]:
df = df.dropna(subset = ['drive_id', 'timestamp', 'lat', 'lon'])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   drive_id   10000 non-null  int64              
 1   timestamp  10000 non-null  datetime64[ns, UTC]
 2   lon        10000 non-null  float64            
 3   lat        10000 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(1)
memory usage: 312.6 KB


In [15]:
df = df.sort_values(by=['drive_id', 'timestamp'])

In [16]:
df.head()

,drive_id,timestamp,lon,lat
33,2,2014-01-31 23:00:10.168741+00:00,41.908130,12.504367
89,2,2014-01-31 23:00:25.774039+00:00,41.908663,12.506601
138,2,2014-01-31 23:00:40.261960+00:00,41.909040,12.507911
191,2,2014-01-31 23:00:56.451894+00:00,41.909040,12.507911
240,2,2014-01-31 23:01:09.895936+00:00,41.909040,12.507911


In [17]:
df['lat_shift'] = df.groupby('drive_id')['lat'].shift()

In [18]:
df['lon_shift'] = df.groupby('drive_id')['lon'].shift()

In [19]:
df['time_diff_h'] = df.groupby('drive_id')['timestamp'].diff().dt.total_seconds() / 3600

In [20]:
df.head()

,drive_id,timestamp,lon,lat,lat_shift,lon_shift,time_diff_h
33,2,2014-01-31 23:00:10.168741+00:00,41.908130,12.504367,NaN,NaN,NaN
89,2,2014-01-31 23:00:25.774039+00:00,41.908663,12.506601,12.504367,41.908130,0.004335
138,2,2014-01-31 23:00:40.261960+00:00,41.909040,12.507911,12.506601,41.908663,0.004024
191,2,2014-01-31 23:00:56.451894+00:00,41.909040,12.507911,12.507911,41.909040,0.004497
240,2,2014-01-31 23:01:09.895936+00:00,41.909040,12.507911,12.507911,41.909040,0.003734


In [21]:
def haversine(lat1, lon1, lat2, lon2):
    R=6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

In [22]:
df['distance_km'] = haversine(df['lat_shift'], df['lon_shift'], df['lat'], df['lon'] )

In [23]:
df['speed_kmh'] = df['distance_km'] / df['time_diff_h']

In [24]:
df.head()

,drive_id,timestamp,lon,lat,lat_shift,lon_shift,time_diff_h,distance_km,speed_kmh
33,2,2014-01-31 23:00:10.168741+00:00,41.908130,12.504367,NaN,NaN,NaN,NaN,NaN
89,2,2014-01-31 23:00:25.774039+00:00,41.908663,12.506601,12.504367,41.908130,0.004335,0.255061,58.840144
138,2,2014-01-31 23:00:40.261960+00:00,41.909040,12.507911,12.506601,41.908663,0.004024,0.151347,37.607138
191,2,2014-01-31 23:00:56.451894+00:00,41.909040,12.507911,12.507911,41.909040,0.004497,0.000000,0.000000
240,2,2014-01-31 23:01:09.895936+00:00,41.909040,12.507911,12.507911,41.909040,0.003734,0.000000,0.000000


In [25]:
df = df[(df['speed_kmh'].isna()) | (df['speed_kmh'] <= 200)]

In [26]:
df.head()

,drive_id,timestamp,lon,lat,lat_shift,lon_shift,time_diff_h,distance_km,speed_kmh
33,2,2014-01-31 23:00:10.168741+00:00,41.908130,12.504367,NaN,NaN,NaN,NaN,NaN
89,2,2014-01-31 23:00:25.774039+00:00,41.908663,12.506601,12.504367,41.908130,0.004335,0.255061,58.840144
138,2,2014-01-31 23:00:40.261960+00:00,41.909040,12.507911,12.506601,41.908663,0.004024,0.151347,37.607138
191,2,2014-01-31 23:00:56.451894+00:00,41.909040,12.507911,12.507911,41.909040,0.004497,0.000000,0.000000
240,2,2014-01-31 23:01:09.895936+00:00,41.909040,12.507911,12.507911,41.909040,0.003734,0.000000,0.000000


In [27]:
df = df.drop(columns=['lat_shift', 'lon_shift', 'time_diff_h', 'distance_km', 'speed_kmh'])

In [29]:
df.to_csv("..\\data\\processed\\gps_traces_clean.csv", index=False)